**Numerical Methods Coursework 1**

In [ ]:
import numpy as np
import timeit

Firstly we define the Kamrad-Ritchken model.

In [ ]:

def create_trinomial_model_kamrad_ritchken(s0, r, dt, sigma, N, lamda):
    #model parameters
    
    u=np.exp(lamda*sigma*np.sqrt(dt))
    qu = 1/(2*lamda**2) + ((r - sigma**2/2)*np.sqrt(dt))/(2*lamda*sigma)
    qm = 1-1/lamda**2
    qd = 1/(2*lamda**2) - ((r - sigma**2/2)*np.sqrt(dt))/(2*lamda*sigma)

    if (qu < 0 or qd < 0):
        print("The value of dt is too large and the probabilities will not be viable, please use a smaller value for dt.")
        return

    # Set up the tree via S[n,k] = s0 * u^(k-n)
    prices = np.zeros((N+1, 2*N+1))
    for t in range(N+1):
        prices[t, 0:2*t+1] = s0 * u**np.arange(-t, t+1, 1)

    return (prices, np.array([qd, qm, qu]), u)

    
        

We create an array of prices for $A^n_j$ noticing that these prices have no dependence on $n$.

In [ ]:
def create_auxillary_values(s0, N, u):
    indices = np.arange(0,N+1)
    AValues = s0*u**indices
    return AValues

Next we define the funtion for generating the prices by backward induction

In [ ]:
def generate_lookback_prices(s0, r, dt, sigma, N, lamda, american = False):
    Nfloor_div_2 = N//2     
    prices, probabilities, u = create_trinomial_model_kamrad_ritchken(s0, r, dt, sigma,  N, lamda)
    auxillary_values = create_auxillary_values(s0, N, u)
            
    # Option values matrix indexed by time, k, j
    option_values = np.zeros([N+1, 2*N+1, Nfloor_div_2+1])

    # Terminal condition: option values given by terminal payoff

    terminal_stock_price = prices[N]
    k_range = np.arange(0,2*N + 1)
    j_range = np.arange(0, Nfloor_div_2+1)

    #find x such that A^N_{k,j} = S_0u**x indexed by k, j
    #Note this can produce negative indexes, but these represent impossible situations
    #so they will not impact any resulting prices.
    auxiliary_variable_index = (k_range//2)[:,np.newaxis] - Nfloor_div_2 + j_range[np.newaxis, :]
    option_values[N] = np.maximum(np.take_along_axis(auxillary_values[np.newaxis,:], auxiliary_variable_index, axis = 1) - terminal_stock_price[:,np.newaxis], np.zeros(1)[:,np.newaxis])
    
            
    # Backward induction

    # Loop in time
    for n in reversed(range(N)):
               
        #generate the possible values of k_new indexed by current k, stock movement where for the stock movement 0 is down, 1 is no movement, 2 is up.
        current_k = np.arange(0,2*n+1)
        k_new = np.column_stack([current_k, current_k+1, current_k+2])
        floor_k_new_div_2 = k_new//2
        floor_k_div_2 = current_k//2
        j_range = np.arange(0,Nfloor_div_2+1)
        alpha = floor_k_div_2[:, np.newaxis, np.newaxis] + j_range[np.newaxis,:,np.newaxis]
        beta = Nfloor_div_2 + k_new[:,np.newaxis, :] - n - 1

        #Use the shooting function to generate the possible values of jnew indexed by current k, current j, stock movement where for the stock movement 0 is down, 1 is no movement, 2 is up.
        phi_result = -1 * floor_k_new_div_2[:,np.newaxis, :] + np.maximum(alpha, beta)

        #calculate the option prices using backward induction
        continuation_values = (probabilities[0]* np.take_along_axis(option_values[n+1, 0:2*n + 1, :], phi_result[:,:,0], axis = 1) + probabilities[1]* np.take_along_axis(option_values[n+1, 1:2*n + 2, :], phi_result[:,:,1], axis = 1) + probabilities[2]* np.take_along_axis(option_values[n+1, 2:2*n + 3, :], phi_result[:,:,2], axis = 1))

        if american:
            j_range = np.arange(0, Nfloor_div_2+1)

            #find x such that A^n_{k,j} = S_0u**x indexed again by k, j
            auxiliary_variable_index = (np.floor(current_k/2)[:,np.newaxis] - Nfloor_div_2 + j_range[np.newaxis, :]).astype('int')
            current_stock_price = prices[n]
            exercise_values = np.maximum(np.take_along_axis(auxillary_values[np.newaxis,:], auxiliary_variable_index, axis = 1) - current_stock_price[0:2*n + 1,np.newaxis], np.zeros(1)[:,np.newaxis])
            option_values[n][0:2*n+1,0:Nfloor_div_2+1] = np.maximum(exercise_values, np.exp(-1*r*dt)*continuation_values)
        else:
            option_values[n][0:2*n+1,0:Nfloor_div_2+1] =  np.exp(-1*r*dt)*continuation_values

    #print the fair price for the initial time
    print(option_values[0,0,Nfloor_div_2])
    return option_values

*Finally let's run the code to obtain the price

In [ ]:
def AsianOptionPricing(s0, r, dt, sigma, N, lamda, american = False):
    Ndiv2 = N//2
    prices, probabilities, u = create_trinomial_model_kamrad_ritchken(s0, r, dt, sigma,  N, lamda)
            
    # Option values matrix
    OptPx = np.zeros([N+1, 2*N+1, Ndiv2+1])
    
    
    # Terminal condition: option values given by terminal payoff
    for k in range(2*N+1):
        j_range = np.arange(0, Ndiv2+1)
        OptPx[N, k, j_range] = np.maximum(s0 * u **(k//2 -N//2 + j_range) - prices[N, k], 0)

    for n in reversed(range(N)):
 
        # Loop across each stock price index k
        for k in range(2*n+1):
            
            j_range = np.arange(0, N//2+1)
            
            alpha = k//2 + j_range
            phi_up = -1 *((k+2)//2) + np.maximum(alpha, N//2 + (k+2) - n - 1)  
            phi_mid = -1 *((k+1)//2) + np.maximum(alpha, N//2 + (k+1) - n - 1)
            phi_down = -1 * (k//2) + np.maximum(alpha, N//2 + k - n - 1)
            
            V_up = OptPx[n+1, k+2, phi_up]
            V_mid = OptPx[n+1, k+1, phi_mid]
            V_down = OptPx[n+1, k, phi_down]
            
            
            
            # the backward induction step

            if american:
                for k in range(2*n+1):
                    j_range = np.arange(0, n+1)
                    OptPx[N, k, j_range] = np.maximum(np.maximum(s0 * u **j_range - prices[n, k], 0), np.exp(-1*r*dt)*(probabilities[2]*V_up + probabilities[1]*V_mid + probabilities[0]*V_down))
            else:
                OptPx[n,k,j_range] = np.exp(-1*r*dt)*(probabilities[2]*V_up + probabilities[1]*V_mid + probabilities[0]*V_down)
            
    print(OptPx[0,0,N//2])
    return OptPx

In [ ]:
s0 = 100
r = 0.1

sigma = 0.1
N = 500
dt = 0.1
lamda = 1.25


In [ ]:


for i in range(10):
    AsianOptionPricing(s0, r, dt, sigma, N, lamda, american=False)


In [ ]:
for i in range(10):
    generate_lookback_prices(s0, r, dt, sigma, N, lamda, american=False)